In [ ]:
!pip install python-telegram-bot gspread oauth2client


In [ ]:
!pip install nest_asyncio


In [ ]:
!pip install python-telegram-bot --upgrade


In [ ]:
# Install packages (run once in Colab)
# !pip install gspread oauth2client python-telegram-bot nest_asyncio

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters
import nest_asyncio

# Enable asyncio inside Colab
nest_asyncio.apply()

# === GOOGLE SHEETS SETUP ===
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/credentials.json", scope)
client = gspread.authorize(creds)
sheet = client.open_by_key("1lfiFzG_xoniU9NcCLK3GQjPgDjW-pOcPv1EluCIVCvo").sheet1  # Replace with your sheet ID

# === BOT COMMANDS ===

# /start command
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Hi! 👋\n"
        "Use this format to add an expense:\n"
        "`Add: Brand, Description, Price, Size, Volume, Classification, PurchasePrice, VendorNumber, VendorName`\n\n"
        "To delete a row, use:\n"
        "`Delete: BrandName`",
        parse_mode="Markdown"
    )

# Handle Add and Delete
# Handle Add and Delete
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = update.message.text.strip()

    if text.lower().startswith("add:"):
        try:
            parts = text[4:].split(",")
            if len(parts) != 9:
                await update.message.reply_text("❌ Please provide exactly 9 values separated by commas.")
                return
            row = [p.strip() for p in parts]
            sheet.append_row(row)
            await update.message.reply_text("✅ Expense added successfully!")
        except Exception as e:
            await update.message.reply_text(f"⚠️ Error: {str(e)}")

    elif text.lower().startswith("delete:"):
        try:
            brand_to_delete = text[7:].strip().lower()
            all_rows = sheet.get_all_values()

            # Skip header (index 0)
            for i in range(1, len(all_rows)):
                row = all_rows[i]
                if row and row[0].strip().lower() == brand_to_delete:
                    sheet.delete_rows(i + 1)  # delete row at index i + 1
                    await update.message.reply_text(f"✅ Row with brand '{row[0]}' deleted.")
                    return

            await update.message.reply_text(f"❌ Brand '{brand_to_delete}' not found.")

        except Exception as e:
            await update.message.reply_text(f"⚠️ Error: {str(e)}")

    else:
        await update.message.reply_text("❓ Unknown format. Use /start for help.")


# === MAIN FUNCTION TO START BOT ===
async def main():
    bot_token = "7415877681:AAErMIs6n0D-Sr235oRcIEL7x7j2GKPJCu8"  # Replace with your real token
    app = ApplicationBuilder().token(bot_token).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("🤖 Bot is running...")
    await app.run_polling()

# === RUN THE BOT ===
await main()


🤖 Bot is running...
